In [182]:
from src.ingestion.database.reader import IngestionReaderInterface
from src.ingestion.database.reader_psql import ConfigurePostgresSparkSession
from src.ingestion.database.reader_psql import PostgresIngestionReader
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, schema_of_json, get_json_object
from pyspark.sql import functions as F
from pyspark.sql import types as T
from src.ingestion.database.reader import ReadContents
from src.ingestion.database.reader import ReadUsers
from src.ingestion.database.reader import ReadRatingFeedbacks
from pyspark.sql.functions import array_contains, col, explode, expr
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.linalg import SparseVector, DenseVector
import json

In [9]:
spark_builder = SparkSession.builder.appName("Jupyter")
spark_builder = ConfigurePostgresSparkSession(spark_builder)
spark = spark_builder.getOrCreate()
reader = PostgresIngestionReader(db_host="127.0.0.1", db_user="minfei", db_password="FM199601060046gg", spark=spark)
content = ReadContents(reader)
user = ReadUsers(reader)
rating = ReadRatingFeedbacks(reader)


In [203]:
# get the original_language of movies
# tmdb = content.select(["id","tmdb_primary_info"])
# lan = tmdb.withColumn('orginal_language',
#                    get_json_object('tmdb_primary_info', '$.original_language'))
# lan.show(10)

#get the original_language of movies
tmdb = content.select(["id","tmdb_primary_info"])
lan = tmdb.withColumn('languages',
                   get_json_object('tmdb_primary_info', '$.spoken_languages'))
newdf= lan.withColumn('spoken_languages_all', F.from_json(
        F.col('languages'),
        T.ArrayType(T.StructType([
            T.StructField('name', T.StringType()),
            T.StructField('iso_639_1', T.StringType()),
            T.StructField('english_name', T.StringType()),
        ]))
    ))
newdf = newdf.select('id','spoken_languages_all')
spoken_languages = newdf.withColumn('languages' , expr("transform(spoken_languages_all, x -> x['english_name'])"))
spoken_languages.select('id','languages')



DataFrame[id: bigint, languages: array<string>]

In [234]:
df_sep.take(5)

[Row(id=191689, l_Abkhazian=0, l_Afar=0, l_Afrikaans=0, l_Akan=0, l_Albanian=0, l_Amharic=0, l_Arabic=0, l_Aragonese=0, l_Armenian=0, l_Assamese=0, l_Avaric=0, l_Aymara=0, l_Azerbaijani=0, l_Bambara=0, l_Bashkir=0, l_Basque=0, l_Belarusian=0, l_Bengali=0, l_Bislama=0, l_Bosnian=0, l_Breton=0, l_Bulgarian=0, l_Burmese=0, l_Cantonese=0, l_Catalan=0, l_Chechen=0, l_Chichewa; Nyanja=0, l_Cornish=0, l_Corsican=0, l_Cree=0, l_Croatian=0, l_Czech=0, l_Danish=0, l_Dutch=0, l_Dzongkha=0, l_English=0, l_Esperanto=0, l_Estonian=0, l_Ewe=0, l_Faroese=0, l_Finnish=0, l_French=0, l_Frisian=0, l_Fulah=0, l_Gaelic=0, l_Galician=0, l_Ganda=0, l_Georgian=0, l_German=0, l_Greek=0, l_Guarani=0, l_Gujarati=0, l_Haitian; Haitian Creole=0, l_Hausa=0, l_Hebrew=0, l_Hindi=0, l_Hungarian=0, l_Icelandic=0, l_Ido=0, l_Igbo=0, l_Indonesian=0, l_Inuktitut=0, l_Inupiaq=0, l_Irish=0, l_Italian=0, l_Japanese=0, l_Javanese=0, l_Kalaallisut=0, l_Kannada=0, l_Kazakh=0, l_Khmer=0, l_Kikuyu=0, l_Kinyarwanda=0, l_Kirghiz=0,

In [242]:
df1 = spoken_languages.select('id','languages')
languages_list = [x[0] for x in df1.select(explode("languages").alias("languages")).distinct().orderBy("languages").collect()]
df_sep = df1.select("*" ,*[
    array_contains("languages", lan).alias("l_{}".format(lan)).cast("integer")
    for lan in languages_list]
).drop('languages')
selected_columns = [column for column in df_sep.columns if column.startswith("l_")] 
assembler = VectorAssembler(inputCols=selected_columns, outputCol='sparse_languages',handleInvalid="skip")

df_sep1 = assembler.transform(df_sep).select('id','sparse_languages').na.fill(value = 0, subset=["sparse_languages"])

df_sep1.take(10)
# def sparse_to_array(v):
#     v = DenseVector(v)
#     new_array = list([float(x) for x in v])
#     return new_array
# sparse_to_array_udf = F.udf(sparse_to_array, T.ArrayType(T.FloatType()))
# res = df_sep.withColumn('languages', sparse_to_array_udf('sparse_languages')).select('id','languages')
# res.first()
# df_sep.show()


[Row(id=191689, sparse_languages=SparseVector(150, {109: 1.0})),
 Row(id=147752, sparse_languages=SparseVector(150, {6: 1.0, 17: 1.0, 49: 1.0, 64: 1.0, 101: 1.0, 133: 1.0, 145: 1.0})),
 Row(id=154984, sparse_languages=SparseVector(150, {64: 1.0})),
 Row(id=132725, sparse_languages=SparseVector(150, {35: 1.0, 122: 1.0})),
 Row(id=174751, sparse_languages=SparseVector(150, {33: 1.0})),
 Row(id=127419, sparse_languages=SparseVector(150, {35: 1.0})),
 Row(id=136824, sparse_languages=SparseVector(150, {})),
 Row(id=25989, sparse_languages=SparseVector(150, {41: 1.0})),
 Row(id=156027, sparse_languages=SparseVector(150, {41: 1.0})),
 Row(id=160325, sparse_languages=SparseVector(150, {23: 1.0}))]

In [250]:
df1 = spoken_languages.select('id','languages')

df1 = spoken_languages.select('id','languages')
languages_list = [x[0] for x in df1.select(explode("languages").alias("languages")).distinct().orderBy("languages").collect()]
df_sep = df1.select("*" ,*[
    array_contains("languages", lan).alias("l_{}".format(lan)).cast("integer")
    for lan in languages_list]
).drop('languages')
selected_columns = [column for column in df_sep.columns if column.startswith("l_")] 
assembler = VectorAssembler(inputCols=selected_columns, outputCol='sparse_languages',handleInvalid="skip")

df_sep1 = assembler.transform(df_sep).select('id','sparse_languages').na.fill(value = 0, subset=["sparse_languages"])

df_sep1.take(20)
def sparse_to_array(v):
    v = DenseVector(v)
    new_array = list([float(x) for x in v])
    return new_array
sparse_to_array_udf = F.udf(sparse_to_array, T.ArrayType(T.FloatType()))
res = df_sep1.withColumn('languages', sparse_to_array_udf('sparse_languages')).select('id','languages')
#res.filter(res['id'] ==  147752) 



In [200]:
content.select(["id","tmdb_primary_info"]).take(1)

[Row(id=191689, tmdb_primary_info='{"id": 399168, "adult": false, "title": "The Mathematician and the Devil", "video": false, "budget": 0, "genres": [{"id": 35, "name": "Comedy"}, {"id": 18, "name": "Drama"}, {"id": 14, "name": "Fantasy"}], "status": "Released", "imdb_id": "tt3154916", "revenue": 0, "runtime": 21, "tagline": "", "homepage": "", "overview": "A mathematician offers to sell his soul to the devil for a proof or disproof of Fermat\'s Last Theorem. Based on \\"The Devil and Simon Flagg\\" by Arthur Porges.", "popularity": 0.6, "vote_count": 6, "poster_path": "/5JCaWtCySRPy2JbHwgUAmYJBM8b.jpg", "release_date": "1972-06-06", "vote_average": 8.3, "backdrop_path": null, "original_title": "Математик и чёрт", "spoken_languages": [{"name": "Pусский", "iso_639_1": "ru", "english_name": "Russian"}], "original_language": "ru", "production_companies": [{"id": 88367, "name": "Centrnauchfilm", "logo_path": "/8BGGqyuaxijzhqzmrgdCINWbPhj.png", "origin_country": "SU"}], "production_countrie

In [6]:
json.loads(df.select(['tags']).take(1)[0]["tags"])[0]

{'tag': 'Bridget Terry', 'timestamp_secs': 1422435184}

In [7]:
rating_df = ReadRatingFeedbacks(reader)

rating_df.where(rating_df["content_id"] == 1).show()

+-------+----------+-------------------+------+
|user_id|content_id|           rated_at|rating|
+-------+----------+-------------------+------+
|    784|         1|1999-10-13 20:34:04|   3.0|
|   1063|         1|1996-05-19 08:56:11|   4.0|
|   1117|         1|2004-11-12 07:42:09|   4.0|
|   1210|         1|2016-03-11 02:17:21|   5.0|
|     67|         1|2017-11-09 22:33:23|   4.0|
|   1888|         1|2006-09-20 23:20:26|   4.5|
|   1502|         1|2015-10-09 21:03:00|   5.0|
|   2571|         1|2016-08-21 14:03:16|   4.0|
|   1064|         1|2007-01-25 04:21:13|   3.5|
|   1887|         1|1997-01-03 06:20:27|   5.0|
|   1718|         1|1996-07-20 20:46:26|   5.0|
|   2357|         1|2015-08-01 23:25:22|   3.0|
|   2301|         1|2015-08-04 22:17:57|   4.0|
|   4452|         1|1996-05-26 23:44:15|   5.0|
|   4849|         1|2008-11-17 04:20:32|   4.0|
|   2160|         1|2015-07-26 17:45:19|   3.0|
|   5206|         1|2000-08-11 15:03:37|   5.0|
|   5410|         1|2009-06-12 18:22:14|

In [126]:
df_sep.show()

+--------------------+
|       sparse_genres|
+--------------------+
|      (20,[5],[1.0])|
|(20,[1,5],[1.0,1.0])|
|      (20,[0],[1.0])|
|      (20,[8],[1.0])|
|(20,[8,9,15],[1.0...|
|      (20,[0],[1.0])|
|(20,[5,9],[1.0,1.0])|
|(20,[1,17],[1.0,1...|
|     (20,[17],[1.0])|
|(20,[1,3,16],[1.0...|
|      (20,[9],[1.0])|
|(20,[3,15],[1.0,1...|
|     (20,[19],[1.0])|
|      (20,[1],[1.0])|
|(20,[5,15],[1.0,1...|
|(20,[1,2,8],[1.0,...|
|(20,[1,2,5,15],[1...|
|(20,[5,8],[1.0,1.0])|
|(20,[8,15],[1.0,1...|
|(20,[6,8,14],[1.0...|
+--------------------+
only showing top 20 rows



In [134]:
df1 = df.select('id','genres')#.filter(df.id == '124861')
# df1.first()['genres'] ##2 7 8
genres_list = [x[0] for x in df1.select(explode("genres").alias("genres")).distinct().orderBy("genres").collect()]
df_sep = df1.select("*" ,*[
    array_contains("genres", g).alias("g_{}".format(g)).cast("integer")
    for g in genres_list]
).drop('genres')
#df_sep.show()
selected_columns = [column for column in df_sep.columns if column.startswith("g_")] 
assembler = VectorAssembler(inputCols=selected_columns, outputCol='sparse_genres')
df_sep = assembler.transform(df_sep).select('id','sparse_genres')
def sparse_to_array(v):
    v = DenseVector(v)
    new_array = list([float(x) for x in v])
    return new_array

sparse_to_array_udf = F.udf(sparse_to_array, T.ArrayType(T.FloatType()))
res = df_sep.withColumn('genres', sparse_to_array_udf('sparse_genres')).select('id','genres')
res.show()



+------+--------------------+
|    id|              genres|
+------+--------------------+
|156777|[0.0, 0.0, 0.0, 0...|
| 26425|[0.0, 1.0, 0.0, 0...|
|146366|[1.0, 0.0, 0.0, 0...|
|  2996|[0.0, 0.0, 0.0, 0...|
|185931|[0.0, 0.0, 0.0, 0...|
|192859|[1.0, 0.0, 0.0, 0...|
| 47774|[0.0, 0.0, 0.0, 0...|
|115887|[0.0, 1.0, 0.0, 0...|
|168774|[0.0, 0.0, 0.0, 0...|
|167762|[0.0, 1.0, 0.0, 1...|
|160696|[0.0, 0.0, 0.0, 0...|
|177939|[0.0, 0.0, 0.0, 1...|
| 77778|[0.0, 0.0, 0.0, 0...|
|181541|[0.0, 1.0, 0.0, 0...|
|127220|[0.0, 0.0, 0.0, 0...|
|151410|[0.0, 1.0, 1.0, 0...|
| 33679|[0.0, 1.0, 1.0, 0...|
| 33603|[0.0, 0.0, 0.0, 0...|
| 92307|[0.0, 0.0, 0.0, 0...|
|155386|[0.0, 0.0, 0.0, 0...|
+------+--------------------+
only showing top 20 rows




[Stage 1203:>                                                       (0 + 1) / 1]



In [ ]:
df1 = content_genres.select('id','cont')
genres_list = [x[0] for x in df1.select(explode("genres").alias("genres")).distinct().orderBy("genres").collect()]
df_sep = df1.select("*", *[
    array_contains("genres", g).alias("g_{}".format(g)).cast("integer")
    for g in genres_list]
    ).drop('genres')
selected_columns = [column for column in df_sep.columns if column.startswith("g_")] 
assembler = VectorAssembler(inputCols=selected_columns, outputCol='sparse_genres')
df_sep = assembler.transform(df_sep).select('id','sparse_genres')
def sparse_to_array(v):
    v = DenseVector(v)
    new_array = list([float(x) for x in v])
    return new_array

sparse_to_array_udf = F.udf(sparse_to_array, T.ArrayType(T.FloatType()))
res = df_sep.withColumn('genres', sparse_to_array_udf('sparse_genres')).select('id','genres')
return res